In [1]:
from pathlib import Path
import numpy as np
import torch, torchvision, json
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss
from efficientnet_pytorch import EfficientNet as EN
from tqdm import tqdm
device="cuda"

In [4]:
data_dir = Path('./data/tiny-imagenet-201/tiny-imagenet-200')
data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
])

In [5]:
valid_set = torchvision.datasets.ImageFolder(data_dir / 'val', data_transforms)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=128,
                                           shuffle=True, num_workers=4, pin_memory=True)
train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128,
                                           shuffle=True, num_workers=4, pin_memory=True)
loss = CrossEntropyLoss()

In [6]:
experiment = 6
seed = 0
model_id = f"{experiment}_{seed}"
model_path = Path(f"outputs/experiment_{experiment}/data/{model_id}").rglob('*pt')
model_path = list(model_path)[0]
model = EN.from_pretrained('efficientnet-b5', num_classes = 200)
model.load_state_dict(torch.load(model_path))
model.to(device)

Loaded pretrained weights for efficientnet-b5


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [7]:
def valid(model, dataset, dataloader, loss, device):
    running_loss = 0.0
    right = 0
    total = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader),
                            total=len(dataset) / dataloader.batch_size):
            data, y = data
            data = data.to(device)
            y = y.to(device)
            ans = model.forward(data)
            los = loss(ans, y)
            running_loss += los.item()
            if i == 77:
                break
                
            right += torch.sum(torch.eq(torch.argmax(ans, dim=1), y))
            total += y.shape[0]
            print(right, total)
    return running_loss, running_loss / len(dataloader.dataset), right/total

In [9]:
train_loss, avg_train, train_acc = valid(model, train_set, train_loader, loss, device)
print(train_loss, avg_train)

  0%|▏                                                                                    | 2/781.25 [00:00<03:37,  3.58it/s]

tensor(35, device='cuda:0') 128
tensor(71, device='cuda:0') 256


  1%|▍                                                                                    | 4/781.25 [00:00<02:45,  4.71it/s]

tensor(105, device='cuda:0') 384
tensor(133, device='cuda:0') 512


  1%|▋                                                                                    | 6/781.25 [00:01<02:28,  5.22it/s]

...

tensor(2450, device='cuda:0') 9472
tensor(2486, device='cuda:0') 9600


 10%|████████▎                                                                           | 77/781.25 [00:13<02:04,  5.65it/s]

tensor(2514, device='cuda:0') 9728
tensor(2551, device='cuda:0') 9856


 10%|████████▎                                                                           | 77/781.25 [00:14<02:08,  5.46it/s]

285.9533064365387 0.0028595330643653868


In [8]:
valid_loss, avg_valid, valid_acc = valid(model, valid_set, valid_loader, loss, device)
print(valid_loss, avg_valid, valid_acc)

  3%|██▏                                                                                  | 2/78.125 [00:01<00:56,  1.34it/s]

tensor(4, device='cuda:0') 128
tensor(8, device='cuda:0') 256


  5%|████▎                                                                                | 4/78.125 [00:02<00:27,  2.74it/s]

tensor(10, device='cuda:0') 384
tensor(12, device='cuda:0') 512


  8%|██████▌                                                                              | 6/78.125 [00:02<00:18,  3.92it/s]

...

 99%|██████████████████████████████████████████████████████████████████████████████████▊ | 77/78.125 [00:15<00:00,  5.61it/s]

tensor(185, device='cuda:0') 9728
tensor(189, device='cuda:0') 9856


 99%|██████████████████████████████████████████████████████████████████████████████████▊ | 77/78.125 [00:15<00:00,  5.05it/s]

589.4906377792358 0.05894906377792358 tensor(0, device='cuda:0')



..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [ ]:
print(valid_acc)